In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [ ]:
INPUT_DIR = "your_input_directory"  
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "community_reports"
ENTITY_TABLE = "entities"
COMMUNITY_TABLE = "communities"
RELATIONSHIP_TABLE = "relationships"
COVARIATE_TABLE = "covariates"
TEXT_UNIT_TABLE = "text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [21]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
community_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")

entities = read_indexer_entities(entity_df, community_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="default-entity-description",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 1320


,id,human_readable_id,title,type,description,text_unit_ids,frequency,degree,x,y
0,0c5c2c9b-523d-418d-b849-7cc6a19421cb,0,P453307765,PREVIOUS BOARD OF DIRECTOR,"P453307765, named FANG LIU, is an individual w...",[edc26110692c720c53dbb8cf9ccb9faa13a5a5a4bcae5...,1,3,0.0,0.0
1,f4871c44-bf21-4796-bda1-8543cf45e234,1,CN,LOCATION,"The entity ""CN"" refers to the ISO country code...",[edc26110692c720c53dbb8cf9ccb9faa13a5a5a4bcae5...,202,201,0.0,0.0
2,4cfcc9c4-5b66-49ff-9ffa-d22bd2c6cbcf,2,IN0010328049,COMPANY,"IN0010328049, also known as Cosco Merchants Pv...",[a4a0fb6768e03c3890e9cfa1d893f93862b89429b1149...,2,3,0.0,0.0
3,ceed1a15-51de-41cb-9ac5-f30130c87fde,3,IN,LOCATION,"The entity ""IN"" represents India, a country wh...",[a4a0fb6768e03c3890e9cfa1d893f93862b89429b1149...,150,150,0.0,0.0
4,b54628aa-9b96-48a4-91d0-cc465bcc23fe,4,P582673405,CURRENT BOARD OF DIRECTOR,"GANGWEI GU, identified by the entity number P5...",[ed5ed488725aeb71d61236bc128ac24abecacfaae95e1...,2,3,0.0,0.0


#### Read relationships

In [22]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
print(relationship_df.head())

Relationship count: 2953
                                     id  human_readable_id        source  \
0  8b3b2a2c-88a5-4dee-84b7-8fe192a60bdf                  0    P453307765   
1  3ded550a-1d13-419e-b14a-6a273f865aea                  1  IN0010328049   
2  c4a002b1-0629-4506-b9b4-335693bde037                  2    P582673405   
3  58ac46e7-9f81-4235-b771-7a6b15a1c1a0                  3   FR908565575   
4  58109ecd-9a4f-4fb2-998d-72648e52c412                  4    CZ01139720   

             target                                        description  \
0                CN              FANG LIU is a national of China (CN).   
1                IN             IN0010328049 is located in India (IN).   
2                CN  P582673405, GANGWEI GU, is a national of China...   
3                FR             FR908565575 is located in France (FR).   
4  DORMANCY HISTORY  The company CZ01139720 is analyzed using the D...   

   weight  combined_degree                                      text_unit

In [20]:
# # NOTE: covariates are turned off by default, because they generally need prompt tuning to be valuable
# # Please see the GRAPHRAG_CLAIM_* settings
# covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

# claims = read_indexer_covariates(covariate_df)

# print(f"Claim records: {len(claims)}")
# covariates = {"claims": claims}

#### Read community reports

In [6]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, community_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 252


,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
0,52f251115070471dacbc290d3041223c,208,208,3,76,[],Palladia Limited and Its Directors,"The community centers around Palladia Limited,...",# Palladia Limited and Its Directors\n\nThe co...,7.5,The impact severity rating is high due to the ...,"[{'explanation': 'Palladia Limited, incorporat...","{\n ""title"": ""Palladia Limited and Its Dire...",2025-03-04,9
1,e2ef2d5a1e7b451480b969009534ffde,209,209,3,76,[],Robert Carl Friedrich Simon and His Corporate ...,The community centers around Robert Carl Fried...,# Robert Carl Friedrich Simon and His Corporat...,6.5,The impact severity rating is moderate to high...,[{'explanation': 'Robert Carl Friedrich Simon ...,"{\n ""title"": ""Robert Carl Friedrich Simon a...",2025-03-04,2
2,101f06ff15e84928bbc1a183a5e647bb,210,210,3,76,[],Waterval Boerdery Beleggings and John Stephen ...,The community centers around Waterval Boerdery...,# Waterval Boerdery Beleggings and John Stephe...,6.5,The impact severity rating is moderate due to ...,[{'explanation': 'Waterval Boerdery Beleggings...,"{\n ""title"": ""Waterval Boerdery Beleggings ...",2025-03-04,2
3,551f039844da4f75884a926104ce20ef,211,211,3,84,[],Corporate Governance Insights: Directors of th...,This community report focuses on the directors...,# Corporate Governance Insights: Directors of ...,8.5,The impact severity rating is high due to the ...,"[{'explanation': 'Chen Ming, also known by the...","{\n ""title"": ""Corporate Governance Insights...",2025-03-04,5
4,6106498dbbe54cdaa9cefab9788940db,212,212,3,84,[],GBFC022923 and Its Directors,"The community centers around GBFC022923, a dis...",# GBFC022923 and Its Directors\n\nThe communit...,7.5,The impact severity rating is high due to the ...,"[{'explanation': 'GBFC022923, also known as Ja...","{\n ""title"": ""GBFC022923 and Its Directors""...",2025-03-04,5


#### Read text units

In [7]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 1430


,id,text,document_ids,n_tokens
0,edc26110692c720c53dbb8cf9ccb9faa13a5a5a4bcae51...,#entities_id: P453307765. #typology_id_type: 1...,[002b0bdfbc351d5ad7eedaaf3837bbddcb653e1f4157e...,96
1,a4a0fb6768e03c3890e9cfa1d893f93862b89429b11491...,#entities_id: IN0010328049. #typology_id_type:...,[003b9817b2df66da8060e809827e6b2a0e730ae94da2d...,159
2,ed5ed488725aeb71d61236bc128ac24abecacfaae95e1f...,#entities_id: P582673405. #typology_id_type: 1...,[008168ec3677047bbd5305a80b7442d051781bee95d5d...,97
3,683a17f9319ee084962a444efa461b5463b5a0c5fe2a6f...,#entities_id: FR908565575. #typology_id_type: ...,[009281cf16e16082d5e152adb57678dc5fac9c0041a77...,148
4,e6175801e5fb7cfcd7dc6d9bd9ba001d403bd35bc94993...,#bvd_id_number: CZ01139720. #methodology_id: 6...,[00dcc74cb38c7aae10bcc8a2809d96fe41c723feb653b...,166


In [ ]:
from graphrag.config.enums import ModelType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.language_model.manager import ModelManager

api_key = "your api key"
llm_model = "gpt-4o-mini"
embedding_model = "text-embedding-3-small"


chat_config = LanguageModelConfig(
    api_key=api_key,
    type=ModelType.OpenAIChat,
    model=llm_model,
    max_retries=20,
)
chat_model = ModelManager().get_or_create_chat_model(
    name="local_search",
    model_type=ModelType.OpenAIChat,
    config=chat_config,
)

token_encoder = tiktoken.encoding_for_model(llm_model)
embedding_config = LanguageModelConfig(
    api_key=api_key,
    type=ModelType.OpenAIEmbedding,
    model=embedding_model,
    max_retries=20,
)

text_embedder = ModelManager().get_or_create_embedding_model(
    name="local_search_embedding",
    model_type=ModelType.OpenAIEmbedding,
    config=embedding_config,
)

### Create local search context builder

In [9]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    # if you did not run covariates during indexing, set this to None
    covariates=None,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [10]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

model_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [11]:
search_engine = LocalSearch(
    model=chat_model,
    context_builder=context_builder,
    token_encoder=token_encoder,
    model_params=model_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [13]:
result = await search_engine.search("Tell me about Palladia Limited.")
print(result.response)

## Overview of Palladia Limited

Palladia Limited, identified by its BvD ID number GB05396445, was a small private limited company incorporated in the United Kingdom on March 17, 2005. The company operated within the banking, insurance, and financial services sector. However, it was dissolved on February 1, 2011, indicating that it is no longer active. At the time of its dissolution, Palladia Limited had one shareholder, one current director, and one previous director, reflecting a streamlined governance structure [Data: Entities (83); Sources (248, 36)].

## Governance and Directorship

The governance of Palladia Limited has been scrutinized due to its atypical directorship patterns. The company was analyzed using the Atypical Directorship methodology, which highlighted potential irregularities in its management structure. Notably, the directors of Palladia Limited had a significant number of inactive directorships, raising concerns about their governance practices and the overall man

In [ ]:
# question = "Tell me about Dr. Jordan Hayes"
# result = await search_engine.search(question)
# print(result.response)

#### Inspecting the context data used to generate the response

In [26]:
result.context_data["entities"].head()
# print(result.context_data["entities"].head())

,id,entity,description,number of relationships,in_context
0,83,GB05396445,"GB05396445, also known as Palladia Limited, is...",11,True
1,1036,GB04263411,"GB04263411, also known as Arladene Limited, is...",4,True
2,835,IN0010686786,"IN0010686786, identified by its BvD ID number,...",7,True
3,424,GB03043883,"GB03043883, also known as Parcolt Limited, is ...",5,True
4,484,IN0000679963,"IN0000679963, also known as Alba Properties & ...",3,True


In [15]:
# result.context_data["relationships"].head()
print(result.context_data["relationships"].head())

     id        source                 target  \
0   560    GB05396445  ATYPICAL DIRECTORSHIP   
1  1555   PA437068RPP  ATYPICAL DIRECTORSHIP   
2  2090    GB03043883  ATYPICAL DIRECTORSHIP   
3  1642    GB04263411  ATYPICAL DIRECTORSHIP   
4  1179  IN0010686786                     IN   

                                         description weight links  in_context  
0  The company GB05396445 is analyzed using the A...    7.0     4        True  
1  The company PA437068RPP is analyzed using the ...    7.0     4        True  
2  The company GB03043883 is analyzed using the A...    7.0     4        True  
3  The company GB04263411 is analyzed using the A...    7.0     4        True  
4             IN0010686786 is located in India (IN).    1.0     3        True  


In [17]:
# result.context_data["sources"].head()
print(result.context_data["sources"].head())

    id                                               text
0  248  #bvd_id_number: GB05396445. #methodology_id: 1...
1   36  #entities_id: GB05396445. #typology_id_type: 0...
2  724  #bvd_id_number: GB04263411. #methodology_id: 1...
3  939  #entities_id: GB04263411. #typology_id_type: 0...
4  626  #bvd_id_number: IN0010686786. #methodology_id:...


In [31]:
# print how many entities, relationships, and reports were included in the context window
print(
    f"Entities: {len(result.context_data['entities'])}, Relationships: {len(result.context_data['relationships'])}, Reports: {len(result.context_data['reports'])}"
)

Entities: 20, Relationships: 37, Reports: 1
